In [1]:
import os
import json
from openai import AzureOpenAI
import dotenv

# Load environment variables from .env file
dotenv.load_dotenv()

    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo" #You must replace this value with the deployment name for your model.
)

In [2]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_cJXWbcIhj8eumnSI3VHuBlKr",
  "created_at": 1710143556,
  "description": null,
  "file_ids": [],
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4-turbo",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ]
}


In [3]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_ZvobsXDOJTAJFbnz0SCPd7nz', created_at=1710143563, metadata={}, object='thread')


In [4]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

In [5]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_NgA8C8GoOXHo3kuyu4kCq3Zj",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1710143568,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "thread_id": "thread_ZvobsXDOJTAJFbnz0SCPd7nz"
    }
  ],
  "object": "list",
  "first_id": "msg_NgA8C8GoOXHo3kuyu4kCq3Zj",
  "last_id": "msg_NgA8C8GoOXHo3kuyu4kCq3Zj",
  "has_more": false
}


In [11]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [16]:
status = run.status
print(status)

queued


In [17]:
import time
from IPython.display import clear_output


def wait_for_response(thread, run):
  start_time = time.time()

  status = run.status

  while status not in ["completed", "cancelled", "expired", "failed"]:
      time.sleep(5)
      run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
      print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
      status = run.status
      print(f'Status: {status}')
      clear_output(wait=True)

  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  ) 
  
  print(f'Status: {status}')
  print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
  print(messages.model_dump_json(indent=2))

In [8]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_NgA8C8GoOXHo3kuyu4kCq3Zj",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1710143568,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "thread_id": "thread_ZvobsXDOJTAJFbnz0SCPd7nz"
    }
  ],
  "object": "list",
  "first_id": "msg_NgA8C8GoOXHo3kuyu4kCq3Zj",
  "last_id": "msg_NgA8C8GoOXHo3kuyu4kCq3Zj",
  "has_more": false
}


In [9]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
print(data['data'][0]['content'][0])


{'text': {'annotations': [], 'value': 'Create a visualization of a sinewave'}, 'type': 'text'}


In [10]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)  # Outputs: assistant-1YGVTvNzc2JXajI5JU9F0HMD

KeyError: 'image_file'

In [19]:
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

In [20]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("sinewave.png")
image.show()

In [21]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the sinewave"
)

In [22]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

queued
